In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split,cross_validate, GridSearchCV
from sklearn.preprocessing import StandardScaler,OrdinalEncoder
from sklearn.metrics import mean_squared_error
from sklearn.impute import IterativeImputer, SimpleImputer, KNNImputer, MissingIndicator
from sklearn.linear_model import LinearRegression,Lasso,Ridge,BayesianRidge
from sklearn.ensemble import GradientBoostingRegressor, RandomForestRegressor
from xgboost import XGBRegressor
from IPython.display import Image
from pandas_profiling import ProfileReport


In [2]:
final_test = pd.read_csv('data/test.csv')
train_init = pd.read_csv('data/train.csv')
train_init = train_init.drop(columns=["Id"])
# train_init.columns
# train_init.head()

In [3]:

labels = train_init["SalePrice"]
features = train_init.drop(columns=["SalePrice"])

# Set up training and testing data
x_train,x_test,y_train,y_test=train_test_split(features, labels, test_size=0.2, random_state=7)

In [42]:
# Define categorical columns
bools = list(x_train.select_dtypes("bool").columns)
print(f"Boolean columns are: {bools}")

# Define categorical columns
categorical = list(x_train.select_dtypes(exclude=["number", "bool"]).columns)
print(f"Categorical columns are: {categorical}")

# Define numerical columns
numerical = list(x_train.select_dtypes('number').columns)
print(f"Numerical columns are: {numerical}")


Boolean columns are: []
Categorical columns are: ['MSZoning', 'Street', 'Alley', 'LotShape', 'LandContour', 'Utilities', 'LotConfig', 'LandSlope', 'Neighborhood', 'Condition1', 'Condition2', 'BldgType', 'HouseStyle', 'RoofStyle', 'RoofMatl', 'Exterior1st', 'Exterior2nd', 'MasVnrType', 'ExterQual', 'ExterCond', 'Foundation', 'BsmtQual', 'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinType2', 'Heating', 'HeatingQC', 'CentralAir', 'Electrical', 'KitchenQual', 'Functional', 'FireplaceQu', 'GarageType', 'GarageFinish', 'GarageQual', 'GarageCond', 'PavedDrive', 'PoolQC', 'Fence', 'MiscFeature', 'SaleType', 'SaleCondition']
Numerical columns are: ['MSSubClass', 'LotFrontage', 'LotArea', 'OverallQual', 'OverallCond', 'YearBuilt', 'YearRemodAdd', 'MasVnrArea', 'BsmtFinSF1', 'BsmtFinSF2', 'BsmtUnfSF', 'TotalBsmtSF', '1stFlrSF', '2ndFlrSF', 'LowQualFinSF', 'GrLivArea', 'BsmtFullBath', 'BsmtHalfBath', 'FullBath', 'HalfBath', 'BedroomAbvGr', 'KitchenAbvGr', 'TotRmsAbvGrd', 'Fireplaces', 'Garage

In [47]:
numerical = x_train.select_dtypes(include='number')
numerical_cols = numerical.columns.to_list()

categorical = x_train.select_dtypes(exclude=["number"]).columns.to_list()
categorical_cols = categorical.columns.to_list()

['MSSubClass',
 'LotFrontage',
 'LotArea',
 'OverallQual',
 'OverallCond',
 'YearBuilt',
 'YearRemodAdd',
 'MasVnrArea',
 'BsmtFinSF1',
 'BsmtFinSF2',
 'BsmtUnfSF',
 'TotalBsmtSF',
 '1stFlrSF',
 '2ndFlrSF',
 'LowQualFinSF',
 'GrLivArea',
 'BsmtFullBath',
 'BsmtHalfBath',
 'FullBath',
 'HalfBath',
 'BedroomAbvGr',
 'KitchenAbvGr',
 'TotRmsAbvGrd',
 'Fireplaces',
 'GarageYrBlt',
 'GarageCars',
 'GarageArea',
 'WoodDeckSF',
 'OpenPorchSF',
 'EnclosedPorch',
 '3SsnPorch',
 'ScreenPorch',
 'PoolArea',
 'MiscVal',
 'MoSold',
 'YrSold']

In [33]:
df = features
# Create a boolean mask indicating the presence of missing values in each column
mask = df.isna()
# Use the sum method to count the number of missing values in each column
na_percent = mask.mean()
sorted_na_percent = na_percent.sort_values(ascending=False)

# Print the column names and percentage of missing values for columns with missing values
print(sorted_na_percent[sorted_na_percent > 0])


Index(['PoolQC', 'MiscFeature', 'Alley', 'Fence', 'FireplaceQu', 'LotFrontage',
       'GarageType', 'GarageYrBlt', 'GarageQual', 'GarageCond', 'GarageFinish',
       'BsmtFinType2', 'BsmtExposure', 'BsmtCond', 'BsmtFinType1', 'BsmtQual',
       'MasVnrArea', 'MasVnrType', 'Electrical', 'BedroomAbvGr',
       'BsmtHalfBath', 'FullBath', 'KitchenAbvGr', 'HalfBath', 'Functional',
       'Fireplaces', 'KitchenQual', 'TotRmsAbvGrd', 'MSSubClass', 'GrLivArea',
       'GarageCars', 'GarageArea', 'PavedDrive', 'WoodDeckSF', 'OpenPorchSF',
       'EnclosedPorch', '3SsnPorch', 'ScreenPorch', 'PoolArea', 'MiscVal',
       'MoSold', 'YrSold', 'SaleType', 'BsmtFullBath', 'HeatingQC',
       'LowQualFinSF', 'Neighborhood', 'OverallCond', 'OverallQual',
       'HouseStyle', 'BldgType', 'Condition2', 'Condition1', 'LandSlope',
       '2ndFlrSF', 'LotConfig', 'Utilities', 'LandContour', 'LotShape',
       'Street', 'LotArea', 'YearBuilt', 'YearRemodAdd', 'RoofStyle',
       'RoofMatl', 'Exterior1st', 

In [9]:
df.fillna('No', inplace=True)
df.head(10).style.background_gradient(cmap = "viridis")
df.describe().transpose().style.background_gradient(cmap = "magma")
# df.info

,count,mean,std,min,25%,50%,75%,max
MSSubClass,1460.000000,56.897260,42.300571,20.000000,20.000000,50.000000,70.000000,190.000000
LotArea,1460.000000,10516.828082,9981.264932,1300.000000,7553.500000,9478.500000,11601.500000,215245.000000
OverallQual,1460.000000,6.099315,1.382997,1.000000,5.000000,6.000000,7.000000,10.000000
OverallCond,1460.000000,5.575342,1.112799,1.000000,5.000000,5.000000,6.000000,9.000000
YearBuilt,1460.000000,1971.267808,30.202904,1872.000000,1954.000000,1973.000000,2000.000000,2010.000000
YearRemodAdd,1460.000000,1984.865753,20.645407,1950.000000,1967.000000,1994.000000,2004.000000,2010.000000
BsmtFinSF1,1460.000000,443.639726,456.098091,0.000000,0.000000,383.500000,712.250000,5644.000000
BsmtFinSF2,1460.000000,46.549315,161.319273,0.000000,0.000000,0.000000,0.000000,1474.000000
BsmtUnfSF,1460.000000,567.240411,441.866955,0.000000,223.000000,477.500000,808.000000,2336.000000
TotalBsmtSF,1460.000000,1057.429452,438.705324,0.000000,795.750000,991.500000,1298.250000,6110.000000


In [10]:
df.describe().transpose().style.background_gradient(cmap = "magma")

,count,mean,std,min,25%,50%,75%,max
MSSubClass,1460.000000,56.897260,42.300571,20.000000,20.000000,50.000000,70.000000,190.000000
LotArea,1460.000000,10516.828082,9981.264932,1300.000000,7553.500000,9478.500000,11601.500000,215245.000000
OverallQual,1460.000000,6.099315,1.382997,1.000000,5.000000,6.000000,7.000000,10.000000
OverallCond,1460.000000,5.575342,1.112799,1.000000,5.000000,5.000000,6.000000,9.000000
YearBuilt,1460.000000,1971.267808,30.202904,1872.000000,1954.000000,1973.000000,2000.000000,2010.000000
YearRemodAdd,1460.000000,1984.865753,20.645407,1950.000000,1967.000000,1994.000000,2004.000000,2010.000000
BsmtFinSF1,1460.000000,443.639726,456.098091,0.000000,0.000000,383.500000,712.250000,5644.000000
BsmtFinSF2,1460.000000,46.549315,161.319273,0.000000,0.000000,0.000000,0.000000,1474.000000
BsmtUnfSF,1460.000000,567.240411,441.866955,0.000000,223.000000,477.500000,808.000000,2336.000000
TotalBsmtSF,1460.000000,1057.429452,438.705324,0.000000,795.750000,991.500000,1298.250000,6110.000000


In [17]:
# var_num = ["SalePrice", "OverallQual", "GrLivArea", "GarageCars", "TotalBsmtSF", "FullBath", "YearBuilt"]
# sns.pairplot(df[var_num]);
len(df)
df_samp = df.sample(100)
df_samp

,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,...,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition
1309,20,RL,NaN,7153,Pave,NaN,Reg,Lvl,AllPub,Inside,...,0,0,NaN,GdWo,NaN,0,6,2008,WD,Normal
1207,20,RL,70.0,9135,Pave,NaN,Reg,Lvl,AllPub,Inside,...,0,0,NaN,NaN,NaN,0,5,2006,WD,Normal
1099,20,RL,82.0,11880,Pave,NaN,IR1,Lvl,AllPub,Inside,...,0,0,NaN,NaN,NaN,0,4,2009,COD,Abnorml
1132,70,RM,90.0,9900,Pave,NaN,Reg,Lvl,AllPub,Inside,...,0,0,NaN,NaN,NaN,0,5,2007,WD,Normal
1341,20,RL,66.0,13695,Pave,NaN,Reg,Lvl,AllPub,Inside,...,0,0,NaN,NaN,NaN,0,7,2008,WD,Normal
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
477,60,RL,105.0,13693,Pave,NaN,Reg,Lvl,AllPub,Inside,...,0,0,NaN,NaN,NaN,0,3,2007,WD,Normal
1385,50,RM,40.0,5436,Pave,NaN,Reg,Lvl,AllPub,Inside,...,0,0,NaN,MnPrv,NaN,0,5,2010,WD,Normal
922,20,RL,65.0,10237,Pave,NaN,Reg,Lvl,AllPub,Inside,...,0,0,NaN,NaN,NaN,0,10,2006,New,Partial
1201,60,RL,80.0,10400,Pave,NaN,Reg,Lvl,AllPub,Corner,...,0,0,NaN,NaN,NaN,0,3,2009,WD,Normal


In [19]:
samp_profile = ProfileReport(df_samp, title="Pandas Profiling Report", minimal=True)
samp_profile.get_description()

Summarize dataset: 100%|██████████| 85/85 [00:00<00:00, 236.09it/s, Completed]                     


{'analysis': {'title': 'Pandas Profiling Report',
  'date_start': datetime.datetime(2023, 1, 16, 9, 23, 51, 29691),
  'date_end': datetime.datetime(2023, 1, 16, 9, 23, 51, 388732),
  'duration': datetime.timedelta(microseconds=359041)},
 'table': {'n': 100,
  'n_var': 79,
  'memory_size': 68168,
  'record_size': 681.68,
  'n_cells_missing': 493,
  'n_vars_with_missing': 16,
  'n_vars_all_missing': 1,
  'p_cells_missing': 0.06240506329113924,
  'types': {'Numeric': 36, 'Categorical': 42, 'Unsupported': 1}},
 'variables': {'MSSubClass': {'n_distinct': 11,
   'p_distinct': 0.11,
   'is_unique': False,
   'n_unique': 1,
   'p_unique': 0.01,
   'type': 'Numeric',
   'hashable': True,
   'value_counts_without_nan': 20     44
   60     19
   50      7
   70      6
   120     6
   80      5
   160     4
   85      3
   190     3
   30      2
   90      1
   Name: MSSubClass, dtype: int64,
   'value_counts_index_sorted': 20     44
   30      2
   50      7
   60     19
   70      6
   80      5

In [ ]:

profile = ProfileReport(df, title="Pandas Profiling Report")
# profile.to_file("profile.html")
# profile.description_set




In [ ]:
profile.get_description()

In [12]:
profile.get_duplicates()

Summarize dataset:  93%|█████████▎| 80/86 [00:01<00:00, 23.04it/s, Calculate auto correlation]     c:\Users\Ruby\anaconda3\lib\site-packages\pandas_profiling\model\correlations.py:67: UserWarning: There was an attempt to calculate the auto correlation, but this failed.
To hide this warning, disable the calculation
(using `df.profile_report(correlations={"auto": {"calculate": False}})`
If this is problematic for your use case, please report this as an issue:
https://github.com/ydataai/pandas-profiling/issues
(include the error message: 'No data; `observed` has size 0.')
  warnings.warn(
Summarize dataset: 100%|██████████| 873/873 [01:48<00:00,  8.06it/s, Completed]                           


,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,# duplicates
